# Libraries and versions

In [1]:
python_version = !python -V #version 3.8.8
import pandas as pd #version 1.2.4
import numpy as np #version 1.20.1
import seaborn as sns #version 0.11.1
import matplotlib as plt #version 3.3.4

#Modelo
from sklearn.metrics import mean_squared_log_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit

#salvar modelo
import pickle

In [2]:
print(f'Python version - {python_version[0][7:]}')
print(f'Pandas version - {pd.__version__}')
print(f'Numpy version - {np.__version__}')
print(f'Seaborn version - {sns.__version__}')
print(f'Matplotlib version - {plt.__version__}')

Python version - 3.8.8
Pandas version - 1.2.4
Numpy version - 1.20.1
Seaborn version - 0.11.1
Matplotlib version - 3.3.4


# Configurations

### Warnings

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Base

In [4]:
bvsp = pd.read_csv('bvsp_modelado_original.csv')
bvsp.head()

,data,abertura,alta,baixa,fechamento,var_abertura,media_movel_180,media_movel_240,media_movel_360,media_movel_480
0,1995-02-27,3217.399902,3345.800049,3184.399902,3270.899902,0.000000,4404.944448,3785.654587,2719.633892,2057.587711
1,1995-02-28,3217.399902,3345.800049,3184.399902,3270.899902,0.000000,4404.830560,3792.938337,2728.090837,2064.238961
2,1995-03-01,3217.399902,3345.800049,3184.399902,3270.899902,0.000000,4404.943892,3800.340836,2736.526114,2070.890836
3,1995-03-02,3270.899902,3270.899902,2970.199951,2989.100098,53.500000,4405.345558,3807.966252,2745.099169,2077.654585
4,1995-03-03,2989.100098,3018.100098,2841.699951,2988.000000,-281.799804,4404.507225,3814.417502,2752.882781,2083.832502


### Transformar índice em data

In [5]:
#transforma coluna em data
bvsp['data'] = pd.to_datetime(bvsp['data'])

#transforma o índice em data
bvsp.set_index('data', inplace=True)

# Importar modelos treinados

In [6]:
#Fechamento
mdl_fechamento = pickle.load(open('mdlfechamento.ridge.v1.sav', 'rb'))

#Baixa
mdl_baixa = pickle.load(open('mdlbaixa.ridge.v1.sav', 'rb'))

#Fechamento
mdl_alta = pickle.load(open('mdlalta.ridge.v1.sav', 'rb'))

## Inserir valores a serem previstos

In [7]:
#inserir os dados de abertura
abertura = float(input('VALOR DE ABERTURA = '))
data = str(input('DATA: '))

#cria DataFrame com dados de entrada
entrada = pd.DataFrame(data={'abertura':[abertura]}, index=[data])

#integra dados ao DataFrame original
dados_entrada = bvsp.append(entrada)

#exclui colunas para adequar ao modelo
dados_entrada.drop(columns=['alta', 'baixa', 'fechamento'], inplace=True)

#cria variáveis estatísticas, Lags e diffs
dados_entrada['var_abertura'] = dados_entrada['abertura'].diff()
dados_entrada['media_movel_180'] = dados_entrada['abertura'].rolling(180).mean()
dados_entrada['media_movel_240'] = dados_entrada['abertura'].rolling(240).mean()
dados_entrada['media_movel_360'] = dados_entrada['abertura'].rolling(360).mean()
dados_entrada['media_movel_480'] = dados_entrada['abertura'].rolling(480).mean()

#exclui valores inválidos
dados_entrada.dropna(inplace=True)

#objeto ajustado para ser inserido no modelo
prever = dados_entrada.iloc[-len(entrada)].to_frame().T

#prever os valores de fechamento, baixa e alta
y_fechamento_previsto = mdl_fechamento.predict(prever)
y_baixa_previsto = mdl_baixa.predict(prever)
y_alta_previsto = mdl_alta.predict(prever)

print('=' * 50)
print('RESULTADOS'.center(50))
print('=' * 50)
print(f'ABERTURA = {abertura}')
print(f'FECHAMENTO = {y_fechamento_previsto[0]:.2f}')
print(f'BAIXA = {y_baixa_previsto[0]:.2f}')
print(f'ALTA = {y_alta_previsto[0]:.2f}')
print('=' * 50)
print('VARIAÇÃO'.center(50))
print('=' * 50)
print(f'ABERTURA X FECHAMENTO = {y_fechamento_previsto[0] - abertura:.2f} / {(1-(abertura / y_fechamento_previsto[0]))*100:.2f}%')
print(f'BAIXA X ALTA = {y_alta_previsto[0] - y_baixa_previsto[0]:.2f} / {(1-(y_baixa_previsto[0] / y_alta_previsto[0]))*100:.2f}%')

VALOR DE ABERTURA =  119392.00
DATA:  2021-04-28


                    RESULTADOS                    
ABERTURA = 119392.0
FECHAMENTO = 119490.61
BAIXA = 118305.49
ALTA = 120518.24
                     VARIAÇÃO                     
ABERTURA X FECHAMENTO = 98.61 / 0.08%
BAIXA X ALTA = 2212.75 / 1.84%


In [8]:
#inserir os dados de abertura
abertura = float(input('Digite o valor de abertura = '))
data = str(input('Digite a data: '))

#cria DataFrame com dados de entrada
entrada = pd.DataFrame(data={'abertura':[abertura]}, index=[data])

#integra dados ao DataFrame original
dados_entrada = bvsp.append(entrada)

#exclui colunas para adequar ao modelo
dados_entrada.drop(columns=['alta', 'baixa', 'fechamento'], inplace=True)

#cria variáveis estatísticas, Lags e diffs
dados_entrada['var_abertura'] = dados_entrada['abertura'].diff()
dados_entrada['media_movel_180'] = dados_entrada['abertura'].rolling(180).mean()
dados_entrada['media_movel_240'] = dados_entrada['abertura'].rolling(240).mean()
dados_entrada['media_movel_360'] = dados_entrada['abertura'].rolling(360).mean()
dados_entrada['media_movel_480'] = dados_entrada['abertura'].rolling(480).mean()

#exclui valores inválidos
dados_entrada.dropna(inplace=True)

#objeto ajustado para ser inserido no modelo
prever = dados_entrada.iloc[-len(entrada)].to_frame().T

#previsão do modelo
modelo.predict(prever)

KeyboardInterrupt: Interrupted by user